In [0]:
%pip install duckdb

In [0]:
import duckdb
import os

import pyspark
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from delta.tables import DeltaTable

#rundate = '20130208_000000'
#rundate = '20130214_000000'

dbutils.widgets.text('rundate', '20130208_000000', 'rundate of the execution')
rundate = dbutils.widgets.get('rundate')

In [0]:
def execute_query(query):

    wal_file = "/Workspace/Users/armando.n90@gmail.com/users_case/local_device/home/mysql/analytics.db.wal"

    if os.path.isfile(wal_file):
        dbutils.fs.rm(wal_file)

    conn = duckdb.connect('/Workspace/Users/armando.n90@gmail.com/users_case/local_device/home/mysql/analytics.db')
    result = conn.execute(query)
    conn.close()

    return result

In [0]:
#DELTA TABLE FOR VISITOR
query = f"""WITH filter AS (
                SELECT DISTINCT Email FROM domain_dev.silver_analytics.audit_visitas a
                WHERE a.metadata_batch_id IN 
                (
                    SELECT batch_id FROM governance_prod.metrics.ingestions 
                    WHERE rundate = '{rundate}' AND catalog_name='domain_dev' AND schema_name='silver_analytics' 
                    AND table_name='audit_visitas'
                )
            )
            SELECT v.* FROM domain_dev.gold_analytics.visitor v
            INNER JOIN filter f ON v.Email = f.Email
         """

output_visitor = spark.sql(query)
output_visitor = output_visitor.toPandas()

In [0]:
#DELTA TABLE FOR STATISTICS
query = f"""WITH filter AS (
                SELECT DISTINCT batch_id FROM governance_prod.metrics.ingestions 
                WHERE rundate = '{rundate}' AND catalog_name='domain_dev' AND schema_name='silver_analytics' 
                    AND table_name='audit_visitas'
            )
            SELECT Email, Jyv, Badmail, Baja, FechaEnvio, FechaOpen, Opens, OpensVirales, FechaClick, Clicks, ClicksVirales,
                Links, IPs, Navegadores, Plataformas, metadata_batch_id 
            FROM domain_dev.silver_analytics.audit_visitas v
            WHERE metadata_batch_id IN (SELECT batch_id FROM filter) AND v.metadata_audit_passed = TRUE
         """

output_statistics = spark.sql(query)
output_statistics = output_statistics.toPandas()

In [0]:
#DELTA TABLE FOR EVENT ERRORS
query = f"""WITH filter AS (
                SELECT DISTINCT batch_id FROM governance_prod.metrics.ingestions 
                WHERE rundate = '{rundate}' AND catalog_name='domain_dev' AND schema_name='silver_analytics' 
                    AND table_name='audit_visitas'
            )
            SELECT *
            FROM governance_prod.metrics.event_errors v
            WHERE batch_id IN (SELECT batch_id FROM filter)
         """

output_event_errors = spark.sql(query)
output_event_errors = output_event_errors.toPandas()

In [0]:
#MERGE INTO VISITOR
query = """MERGE INTO visitor AS target USING output_visitor AS source
           ON target.Email = source.Email
           WHEN MATCHED THEN
                UPDATE SET
                    fechaPrimeraVisita = source.fechaPrimeraVisita,
                    fechaUltimaVisita = source.fechaUltimaVisita,
                    visitasTotales = source.visitasTotales,
                    visitasAnioActual = source.visitasAnioActual,
                    visitasMesActual = source.visitasMesActual
           WHEN NOT MATCHED THEN INSERT
        """

execute_query(query)

In [0]:
#MERGE INTO STATISTICS
query = """MERGE INTO statistic AS target USING output_statistics AS source
           ON target.Email = source.Email AND target.metadata_batch_id = source.metadata_batch_id
           WHEN MATCHED THEN
                UPDATE SET
                    Jyv = source.Jyv,
                    Badmail = source.Badmail,
                    Baja = source.Baja,
                    FechaEnvio = source.FechaEnvio,
                    Opens = source.Opens,
                    OpensVirales = source.OpensVirales,
                    Clicks = source.Clicks,
                    ClicksVirales = source.ClicksVirales,
                    Links = source.Links,
                    IPs = source.IPs,
                    Navegadores = source.Navegadores,
                    Plataformas = source.Plataformas
           WHEN NOT MATCHED THEN INSERT
        """

execute_query(query)

In [0]:
#MERGE INTO EVENT ERRORS
query = """MERGE INTO event_errors AS target USING output_event_errors AS source
           ON target.error_event_id = source.error_event_id
           WHEN MATCHED THEN
                UPDATE SET
                    batch_id = source.batch_id,
                    screen_code = source.screen_code,
                    catalog_name = source.catalog_name,
                    schema_name = source.schema_name,
                    table_name = source.table_name,
                    column_name = source.column_name,
                    record_identifier = source.record_identifier,
                    original_value = source.original_value,
                    replaced_value = source.replaced_value,
                    error_condition = source.error_condition
           WHEN NOT MATCHED THEN INSERT
        """

execute_query(query)